## Step 1: Training the Models

In [1]:
!mkdir -p {HOME}/weights
!wget -P {HOME}/weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10n.pt
!wget -P {HOME}/weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10s.pt
!wget -P {HOME}/weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10b.pt
!wget -P {HOME}/weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10x.pt
!ls -lh {HOME}/weights

total 514304
-rw-r--r--  1 richardjiang  staff    79M May 23 17:38 yolov10b.pt
-rw-r--r--  1 richardjiang  staff    11M Sep 14 23:38 yolov10n.pt
-rw-r--r--  1 richardjiang  staff    31M Sep 14 23:38 yolov10s.pt
-rw-r--r--  1 richardjiang  staff   122M May 23 17:38 yolov10x.pt


In [2]:
%pip install huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [13]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import os

# Initialize the four YOLO models
yolov10_models = {
    "yolov10n": YOLO(f'BASE_WEIGHTS/weights/yolov10n.pt'),
    "yolov10s": YOLO(f'BASE_WEIGHTS/weights/yolov10s.pt'),
    "yolov10x": YOLO(f'BASE_WEIGHTS/weights/yolov10x.pt'),
    "yolov10b": YOLO(f'BASE_WEIGHTS/weights/yolov10b.pt')
}

# Path to the dataset configuration file (YAML)
DATASET_CONFIG = 'trafic_data/data_1.yaml'

/Users/richardjiang/Downloads/2 - Competitions & Research/Microsoft AI Talent Program/Eco-Sight/.venv/lib/python3.12/site-packages/ultralytics/nn/tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related

yolov10n structure: 
YOLOv10DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiL

## 1.1 用Train (没Val的labels) 来测试BASE权重的Accuracy和Loss (Prediction Mode)

In [14]:
for model_name, base_weights in yolov10_models.items():
    # Validate the model
    metrics = base_weights.val(
        data=DATASET_CONFIG,
        imgsz=640,
        batch=-1,
        save_json=True,
        plots=True,
        device=0,
    )
    metrics.box.map 
    metrics.box.map50 
    metrics.box.map75 
    metrics.box.maps

NameError: name 'DATASET_CONFIG' is not defined

## 1.2 用Train图片数据来训练和评估模型 (Training and Evaluation Mode)

### 看backbone layers

In [ ]:
# # Check # of backbone layers for each Yolov10 Model
# for model_name, model_weights in yolov10_models.items():
#     print(f"{model_name} structure: ")
#     print(model_weights.model)  # All models # of backbone 
#     print("\n\n")

In [ ]:
# Directory to save the training graphs for all models
main_folder = 'new-training-graphs'
if not os.path.exists(main_folder):  # 自动帮我们创立好文件夹
    os.makedirs(main_folder)  # 每个模型的文件里面总共两个图（一个accuracy，一个loss）

# Number of epochs to wait for loss improvement before stopping
PATIENCE = 5
metrics = dict()

def train_models():
    # Train each of the four models
    for model_name, model_weights in yolov10_models.items():
        print(f"Training {model_name}...")
        
        # Train the model
        results = model_weights.train(
            data=DATASET_CONFIG,   # path to the YAML config file
            epochs=20,             # number of training epochs
            batch=256,             # batch size
            imgsz=640,             # image size for training (640x640)
            name=model_name,       # name for this experiment run
            workers=2,             # number of data loading workers
            plots=True,            # save loss and mAP plots
            lr0=0.01,              # learning-rate
            patience=PATIENCE,     # Early stopping: stop if no improvement in 5 epochs
            device=0,              # 0: GPU, 1: CPU
            save=True,             # Checkpoints: saves the model at each improvement
            freeze=11,             # Transfer Learning
            verbose=True,          # 可视化训练结果
        )
        precision, recall = results.results_dict['metrics/precision(B)'], results.results_dict['metrics/recall(B)']
        mAP50, mAP50_95 = results.results_dict['metrics/mAP50(B)'], results.results_dict['metrics/mAP50-95(B)']
        epochs = range(1, 21)  # Adjust according to your number of epochs
        
        f1_score = 2 * (precision * recall) / (precision + recall)
        f1_scores = [f1_score] * len(epochs)

        metrics[model_name] = [mAP50, mAP50_95, f1_scores, epochs]

train_models()
print("All models have been trained and metrics saved.")

## 1.3 画F1-Score和Accuracy图表

In [ ]:
def plotting():
    for model_name, metric in metrics.items():

        # Directory for saving this model's training graphs
        model_folder = os.path.join(main_folder, model_name)
        if not os.path.exists(model_folder):
            os.makedirs(model_folder)

        mAP50, mAP50_95 = metric[model_name][mAP50], metric[model_name][mAP50_95]        
        f1_scores, epochs = metric[model_name][f1_scores], metric[model_name][epochs]

        # F1-Score Plot
        plt.figure(figsize=(10, 6))
        plt.plot(epochs, f1_scores, label='F1-Score')
        plt.title(f'{model_name} - F1-Score Over Epochs')
        plt.xlabel('Epochs')
        plt.ylabel('F1-Score')
        plt.legend()
        plt.grid(True)
        f1_score_file_path = os.path.join(model_folder, f"{model_name}_f1_score.png")
        plt.savefig(f1_score_file_path)
        plt.close()
        print(f"F1-Score graph saved at {f1_score_file_path}")

        # mAP50 and mAP50-95 Plot
        plt.figure(figsize=(10, 6))
        plt.plot(epochs, [mAP50]*len(epochs), label='mAP50 (B)')
        plt.plot(epochs, [mAP50_95]*len(epochs), label='mAP50-95 (B)')
        plt.title(f'{model_name} - Accuracy (mAPs) Over Epochs')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy (mAPs)')
        plt.legend()
        plt.grid(True)
        map_file_path = os.path.join(model_folder, f"{model_name}_map.png")
        plt.savefig(map_file_path)
        plt.close()
        print(f"Accuracy graphs saved at {map_file_path}")  

plotting()
print(f"Plotting completed for all and plots saved in {main_folder}.")